# 🎙️ AI Speech Coach - Colab 啟動器

## 📝 使用步驟
### 第一步：設定 OpenAI API Key
**只需要這一個 Key！** 選擇以下任一方式：

- **方式 A（推薦）**：使用 Colab Secrets 
  - 點擊左側 🔑 圖示 → Secrets → 新增 secret
  - Name: `GPT_API_KEY`
  - Value: 貼上你的 OpenAI API key
  - 勾選 "允許 notebook 訪問"
  
- **方式 B**：執行第 2A 格設定環境變數

- **方式 C**：在 Streamlit 介面手動輸入

---

### 第二步：依序執行格子

1. **第 2B 格**：安裝必要套件
2. **第 3 格**：建立 streamlit_app.py
3. **第 4 格**：啟動應用
4. **第 5格**：ngrok 公開連結（需要免費註冊，但更穩定可分享）

In [9]:
# 2B. 安裝必要套件
!pip install -q streamlit openai plotly pydub
!apt-get install -y ffmpeg > /dev/null 2>&1

In [ ]:
# 2A. [可選] 設定 OpenAI API Key（環境變數方式）
# 如果你已經使用 Colab Secrets，可以跳過這格

import os
from getpass import getpass

# 檢查是否已從 Colab Secrets 取得
try:
    from google.colab import userdata
    if userdata.get('GPT_API_KEY'):
        print("✅ 已從 Colab Secrets 讀取到 API Key")
        print("   無需再次設定！")
        os.environ['OPENAI_API_KEY'] = userdata.get('GPT_API_KEY')
    else:
        raise KeyError
except:
    # 手動設定環境變數
    api_key = getpass("請輸入你的 OpenAI API Key: ")
    os.environ['OPENAI_API_KEY'] = api_key
    print("✅ API Key 已設定為環境變數")
    print("   後續使用都不需要再輸入！")

In [ ]:
%%writefile streamlit_app.py
import streamlit as st
import os
from openai import OpenAI
import plotly.graph_objects as go
import json
import base64
from io import BytesIO
from pydub import AudioSegment

# --- Page Config ---
st.set_page_config(
    page_title="AI Speech Coach (Adaptive)",
    layout="wide",
    initial_sidebar_state="expanded"
)

# --- Custom CSS ---
st.markdown("""
<style>
    /* 主標題樣式 */
    .main h1 {
        color: #1E3A8A;
        font-size: 2.5rem !important;
        font-weight: 700;
        margin-bottom: 0.5rem;
    }
    
    /* 上傳區域美化 */
    [data-testid="stFileUploader"] {
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        padding: 2rem;
        border-radius: 15px;
        border: none;
    }
    
    [data-testid="stFileUploader"] label {
        color: white !important;
        font-size: 1.1rem !important;
        font-weight: 600;
    }
    
    /* 側邊欄美化 */
    [data-testid="stSidebar"] {
        background: linear-gradient(180deg, #f8fafc 0%, #e2e8f0 100%);
    }
    
    /* 按鈕美化 */
    .stButton > button {
        width: 100%;
        background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
        color: white;
        border: none;
        padding: 0.75rem 1.5rem;
        font-size: 1.1rem;
        font-weight: 600;
        border-radius: 10px;
        box-shadow: 0 4px 6px rgba(0,0,0,0.1);
        transition: all 0.3s;
    }
    
    .stButton > button:hover {
        transform: translateY(-2px);
        box-shadow: 0 6px 12px rgba(0,0,0,0.15);
    }
    
    /* 指標卡片美化 */
    [data-testid="stMetricValue"] {
        font-size: 2rem;
        font-weight: 700;
        color: #1E3A8A;
    }
    
    /* Tab 美化 */
    .stTabs [data-baseweb="tab-list"] {
        gap: 8px;
    }
    
    .stTabs [data-baseweb="tab"] {
        border-radius: 8px 8px 0 0;
        padding: 0.5rem 1rem;
        font-weight: 600;
    }
    
    /* 進度條美化 */
    .stProgress > div > div {
        background: linear-gradient(90deg, #667eea 0%, #764ba2 100%);
    }
</style>
""", unsafe_allow_html=True)

# --- API Key 智能偵測 ---
def get_api_key():
    """三層優先級自動偵測 API Key"""
    # 1. 優先從 Colab Secrets 讀取
    try:
        from google.colab import userdata
        key = userdata.get('GPT_API_KEY')
        if key:
            return key, "Colab Secrets"
    except:
        pass
    
    # 2. 其次從環境變數讀取
    key = os.getenv('OPENAI_API_KEY')
    if key:
        return key, "環境變數"
    
    # 3. 最後從 session_state 讀取（手動輸入）
    if 'api_key' in st.session_state and st.session_state.api_key:
        return st.session_state.api_key, "手動輸入"
    
    return None, None

api_key, key_source = get_api_key()

# --- Sidebar: Settings ---
st.sidebar.title("⚙️ 演講情境設定")

# API Key 狀態顯示
with st.sidebar.expander("🔑 API Key 設定", expanded=(not api_key)):
    if api_key:
        st.success(f"✅ 已設定（來源：{key_source}）")
    else:
        st.warning("⚠️ 尚未設定 API Key")
        manual_key = st.text_input(
            "請輸入 OpenAI API Key",
            type="password",
            help="或使用 Colab Secrets/環境變數設定",
            key="api_key_input"
        )
        if manual_key:
            st.session_state.api_key = manual_key
            st.rerun()

st.sidebar.markdown("---")
st.sidebar.markdown("### 🎭 演講者設定")

personality = st.sidebar.radio(
    "性格類型",
    options=["I 型 (內向/沉穩)", "E 型 (外向/熱情)"],
    index=0,
    help="選擇你的性格類型以獲得更貼近的建議"
)

st.sidebar.markdown("---")
st.sidebar.markdown("### 🎯 分析設定")

coaching_style = st.sidebar.selectbox(
    "教練風格",
    options=["溫柔鼓勵 (Supportive)", "平衡回饋 (Balanced)", "嚴格魔鬼教練 (Strict/Critical)"],
    index=1,
    help="選擇你希望的回饋風格"
)

scenario = st.sidebar.selectbox(
    "演講場景",
    options=[
        "學位口試/課堂報告 (Thesis Defense/Class Report)",
        "科普演講/公眾推廣 (Public Outreach)",
        "學術研討會 (Conference Presentation)"
    ],
    index=0,
    help="不同場景會有不同的評估重點"
)

audience = st.sidebar.text_input(
    "預設聽眾",
    value="Professors and Graduate Students",
    help="例如: Professors, General Public, High School Students"
)

st.sidebar.markdown("---")
st.sidebar.caption("💡 **提示**: 所有設定都會影響 AI 的分析角度")

# --- Main Area ---
st.title("🎙️ AI 專業演講教練")
st.markdown("""
<div style='background: linear-gradient(135deg, #667eea15 0%, #764ba215 100%); 
            padding: 1rem; border-radius: 10px; margin-bottom: 1.5rem;'>
    <p style='margin: 0; color: #4B5563; font-size: 1.1rem;'>
        🤖 使用 <strong>GPT-4o Audio</strong> 模型進行智能分析 <br>
        📱 支援 iPhone 錄音（M4A）、MP3、WAV 格式
    </p>
</div>
""", unsafe_allow_html=True)

# --- File Upload Section ---
col1, col2 = st.columns([2, 1])

with col1:
    uploaded_file = st.file_uploader(
        "📤 上傳您的演講錄音", 
        type=["mp3", "wav", "m4a"],
        help="支援 MP3、WAV、M4A 格式，M4A 會自動轉換",
        label_visibility="collapsed"
    )

with col2:
    st.markdown("""
    <div style='padding: 1rem; background: #F3F4F6; border-radius: 10px; height: 100%;'>
        <p style='margin: 0; font-size: 0.9rem; color: #6B7280;'>
            <strong>✨ 快速提示</strong><br>
            • 建議錄音長度：1-10 分鐘<br>
            • 檔案大小：< 25 MB<br>
            • 清晰度越高越準確
        </p>
    </div>
    """, unsafe_allow_html=True)

st.markdown("<br>", unsafe_allow_html=True)

# --- Analysis Logic ---

def convert_to_mp3(audio_data, source_format):
    """將音訊轉換為 MP3 格式"""
    try:
        # 載入音訊
        audio = AudioSegment.from_file(BytesIO(audio_data), format=source_format)
        
        # 轉換為 MP3
        mp3_buffer = BytesIO()
        audio.export(mp3_buffer, format="mp3", bitrate="128k")
        mp3_buffer.seek(0)
        
        return mp3_buffer.getvalue()
    except Exception as e:
        raise Exception(f"音訊轉換失敗: {str(e)}")

def encode_audio(file, file_ext):
    """將音訊檔案編碼為 base64，並返回格式和 base64 資料"""
    audio_data = file.getvalue()
    
    # 判斷是否需要轉換
    if file_ext == "m4a":
        # M4A 需要轉換為 MP3（OpenAI API 只支援 mp3 和 wav）
        audio_data = convert_to_mp3(audio_data, "m4a")
        audio_format = "mp3"
    elif file_ext == "mp3":
        audio_format = "mp3"
    else:  # wav
        audio_format = "wav"
    
    # 編碼為 base64
    audio_base64 = base64.b64encode(audio_data).decode("utf-8")
    
    return audio_base64, audio_format

def analyze_audio(client, audio_base64, audio_format, settings):
    """使用 OpenAI GPT-4o Audio API 分析音訊"""
    
    # 1. Personality & Style
    style_instruction = ""
    if "Supportive" in settings['style']:
        style_instruction = "You are a warm, encouraging coach. Focus on potential and strengths mainly. Use gentle language."
    elif "Strict" in settings['style']:
        style_instruction = "You are a very strict, critical coach. Focus on logic holes, weak arguments, and mistakes. Be direct."
    else:  # Balanced
        style_instruction = "You are a balanced professional coach. Provide diverse feedback between pros and cons."

    # 2. Scenario specific logic
    opening_instruction = ""
    if "Thesis Defense" in settings['scenario']:
        opening_instruction = "For the 'Opening Analysis', focus heavily on **Structure & Problem Statement**. Did the speaker clearly define the research gap? Is the outline logical?"
    elif "Public Outreach" in settings['scenario']:
        opening_instruction = "For the 'Opening Analysis', focus heavily on **The Hook**. Did they grab attention immediately? Was it engaging/fun?"
    else:  # Conference
        opening_instruction = "For the 'Opening Analysis', focus on professional delivery and clarity of contribution."

    # 3. Audience check
    audience_instruction = f"Check if the terminology density is appropriate for this audience: {settings['audience']}."

    system_prompt = f"""
You are an expert Speech Coach. Output JSON only.
Language: Traditional Chinese (繁體中文) for all user-facing content.

Settings:
- Coach Personality: {settings['personality']}
- Style: {style_instruction}
- Scenario: {settings['scenario']} ({opening_instruction})
- Audience: {settings['audience']} ({audience_instruction})

Analyze the provided audio. Return a JSON object with this exact structure:
{{
    "summary": {{
        "score": <int 0-100>,
        "wpm": <int>,
        "confidence_level": "<string e.g. High/Medium/Low>"
    }},
    "radar_data": {{
        "professionalism": <int 0-10>,
        "logic_structure": <int 0-10>,
        "vocal_expression": <int 0-10>,
        "time_management": <int 0-10>,
        "audience_fit": <int 0-10>
    }},
    "opening_analysis": {{
        "title": "<string based on scenario e.g. 研究動機與架構清晰度 or 開場吸引力>",
        "content": "<string analysis of the first minute>"
    }},
    "feedback_tabs": {{
        "strengths": ["<string point 1>", "<string point 2>", ...],
        "improvements": ["<string point 1>", "<string point 2>", ...],
        "simulated_qa": ["<string question 1>", "<string question 2>"]
    }}
}}
"""

    try:
        completion = client.chat.completions.create(
            model="gpt-4o-audio-preview",
            modalities=["text"],
            messages=[
                {
                    "role": "system",
                    "content": system_prompt
                },
                {
                    "role": "user",
                    "content": [
                        { 
                            "type": "input_audio", 
                            "input_audio": { 
                                "data": audio_base64, 
                                "format": audio_format
                            }
                        }
                    ]
                }
            ]
        )
        return json.loads(completion.choices[0].message.content)
    except json.JSONDecodeError as e:
        return {"error": f"JSON 解析錯誤: {str(e)}"}
    except Exception as e:
        return {"error": f"API 呼叫失敗: {str(e)}"}

# --- Execution ---

if uploaded_file is not None:
    # 顯示檔案資訊
    file_size_mb = uploaded_file.size / (1024 * 1024)
    file_ext = uploaded_file.name.split('.')[-1].lower()
    
    # 美化的檔案資訊顯示
    col_info1, col_info2, col_info3 = st.columns(3)
    
    with col_info1:
        st.markdown(f"""
        <div style='background: #EEF2FF; padding: 1rem; border-radius: 10px; text-align: center;'>
            <p style='margin: 0; color: #6366F1; font-size: 0.9rem;'>📁 檔案名稱</p>
            <p style='margin: 0; color: #1E3A8A; font-weight: 600;'>{uploaded_file.name}</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col_info2:
        st.markdown(f"""
        <div style='background: #FEF3C7; padding: 1rem; border-radius: 10px; text-align: center;'>
            <p style='margin: 0; color: #D97706; font-size: 0.9rem;'>📊 檔案大小</p>
            <p style='margin: 0; color: #92400E; font-weight: 600;'>{file_size_mb:.2f} MB</p>
        </div>
        """, unsafe_allow_html=True)
    
    with col_info3:
        format_emoji = "🎵" if file_ext == "mp3" else ("📱" if file_ext == "m4a" else "🔊")
        format_text = "MP3" if file_ext == "mp3" else ("M4A → MP3" if file_ext == "m4a" else "WAV")
        st.markdown(f"""
        <div style='background: #DCFCE7; padding: 1rem; border-radius: 10px; text-align: center;'>
            <p style='margin: 0; color: #16A34A; font-size: 0.9rem;'>{format_emoji} 格式</p>
            <p style='margin: 0; color: #166534; font-weight: 600;'>{format_text}</p>
        </div>
        """, unsafe_allow_html=True)
    
    st.markdown("<br>", unsafe_allow_html=True)
    
    if st.button("🚀 開始全方位分析", type="primary", use_container_width=True):
        if not api_key:
            st.error("❌ 請先設定 OpenAI API Key（點擊左側側邊欄的 🔑 API Key 設定）")
        else:
            # 進度條
            progress_bar = st.progress(0)
            status_text = st.empty()
            
            try:
                # Step 1: 準備音訊
                status_text.text("🔄 步驟 1/3：準備音訊檔案...")
                progress_bar.progress(10)
                
                client = OpenAI(api_key=api_key)
                
                # Step 2: 轉換格式（如需要）
                if file_ext == "m4a":
                    status_text.text("🔄 步驟 2/3：轉換 M4A 為 MP3 格式...")
                    progress_bar.progress(30)
                else:
                    status_text.text("🔄 步驟 2/3：處理音訊檔案...")
                    progress_bar.progress(30)
                
                audio_b64, audio_format = encode_audio(uploaded_file, file_ext)
                progress_bar.progress(50)
                
                # Step 3: AI 分析
                status_text.text("🤖 步驟 3/3：AI 正在深度分析您的演講...")
                progress_bar.progress(60)
                
                settings = {
                    "personality": personality,
                    "style": coaching_style,
                    "scenario": scenario,
                    "audience": audience
                }
                
                result = analyze_audio(client, audio_b64, audio_format, settings)
                progress_bar.progress(100)
                
                # 清除進度顯示
                progress_bar.empty()
                status_text.empty()
                
                if "error" in result:
                    st.error(f"""
                    ❌ **分析過程發生錯誤**
                    
                    {result['error']}
                    
                    💡 **建議解決方法**：
                    - 檢查 API Key 是否正確
                    - 確認音訊檔案未損壞
                    - 嘗試重新上傳檔案
                    """)
                else:
                    # --- Display Results ---
                    
                    # 1. Summary Metrics
                    st.success("✅ 分析完成！以下是您的演講評估報告")
                    st.markdown("---")
                    st.markdown("### 📊 整體評估")
                    
                    c1, c2, c3 = st.columns(3)
                    c1.metric("總體評分", f"{result['summary']['score']} 分", delta=None)
                    c2.metric("語速", f"{result['summary']['wpm']} WPM", delta=None)
                    c3.metric("自信度", result['summary']['confidence_level'], delta=None)
                    
                    st.markdown("---")
                    
                    # 2. Radar Chart
                    col_chart, col_opening = st.columns([1, 1])
                    
                    with col_chart:
                        st.subheader("📈 五維雷達分析")
                        radar_data = result['radar_data']
                        categories = ['專業度', '邏輯架構', '語氣表達', '時間掌控', '聽眾適配度']
                        values = [
                            radar_data['professionalism'],
                            radar_data['logic_structure'],
                            radar_data['vocal_expression'],
                            radar_data['time_management'],
                            radar_data['audience_fit']
                        ]
                        
                        fig = go.Figure(data=go.Scatterpolar(
                            r=values,
                            theta=categories,
                            fill='toself',
                            line=dict(color='#667eea', width=3),
                            fillcolor='rgba(102, 126, 234, 0.3)'
                        ))
                        fig.update_layout(
                            polar=dict(
                                radialaxis=dict(
                                    visible=True,
                                    range=[0, 10],
                                    tickfont=dict(size=12)
                                ),
                                angularaxis=dict(
                                    tickfont=dict(size=13, color='#1E3A8A')
                                )
                            ),
                            showlegend=False,
                            height=400,
                            margin=dict(l=80, r=80, t=40, b=40)
                        )
                        st.plotly_chart(fig, use_container_width=True)
                        
                    with col_opening:
                        st.subheader(f"🔍 {result['opening_analysis']['title']}")
                        st.info(result['opening_analysis']['content'])
                    
                    st.markdown("---")
                    
                    # 3. Detailed Tabs
                    tab1, tab2, tab3 = st.tabs(["🌟 優點 (Strengths)", "💡 改進建議 (Improvements)", "❓ 模擬提問 (Simulated Q&A)"])
                    
                    with tab1:
                        st.markdown("### 演講優點")
                        for i, item in enumerate(result['feedback_tabs']['strengths'], 1):
                            st.success(f"**{i}.** {item}")
                            
                    with tab2:
                        st.markdown("### 可改進之處")
                        for i, item in enumerate(result['feedback_tabs']['improvements'], 1):
                            st.warning(f"**{i}.** {item}")
                            
                    with tab3:
                        st.markdown("### 模擬教授/聽眾提問")
                        st.caption("這些是根據您的演講內容，可能會被提問的問題：")
                        for i, q in enumerate(result['feedback_tabs']['simulated_qa'], 1):
                            with st.expander(f"❓ 問題 {i}", expanded=(i==1)):
                                st.markdown(f"{q}")
                            
            except Exception as e:
                progress_bar.empty()
                status_text.empty()
                st.error(f"""
                ❌ **發生未預期的錯誤**
                
                錯誤詳情：{str(e)}
                
                💡 **建議**：
                - 請重新整理頁面再試一次
                - 確認網路連線正常
                - 如問題持續，請檢查 API Key 設定
                """)
else:
    # 未上傳檔案的提示
    st.markdown("""
    <div style='background: linear-gradient(135deg, #FEF3C7 0%, #FDE68A 100%); 
                padding: 2rem; border-radius: 15px; text-align: center; margin-top: 2rem;'>
        <h3 style='color: #92400E; margin-bottom: 1rem;'>👆 請上傳音訊檔案開始分析</h3>
        <p style='color: #78350F; margin: 0; font-size: 1.1rem;'>
            支援 <strong>MP3</strong>、<strong>WAV</strong>、<strong>M4A (iPhone)</strong> 格式<br>
            建議錄音長度：1-10 分鐘，檔案大小 < 25 MB
        </p>
    </div>
    """, unsafe_allow_html=True)
    
    # 使用示例
    st.markdown("---")
    st.markdown("### 💡 使用範例")
    
    ex_col1, ex_col2, ex_col3 = st.columns(3)
    
    with ex_col1:
        st.markdown("""
        <div style='background: #F3F4F6; padding: 1.5rem; border-radius: 10px; height: 100%;'>
            <h4 style='color: #1E3A8A; margin-top: 0;'>🎓 學位口試</h4>
            <p style='color: #6B7280; font-size: 0.9rem;'>
                分析研究動機、架構清晰度、專業術語使用是否恰當
            </p>
        </div>
        """, unsafe_allow_html=True)
    
    with ex_col2:
        st.markdown("""
        <div style='background: #F3F4F6; padding: 1.5rem; border-radius: 10px; height: 100%;'>
            <h4 style='color: #1E3A8A; margin-top: 0;'>🌍 科普演講</h4>
            <p style='color: #6B7280; font-size: 0.9rem;'>
                評估開場吸引力、內容趣味性、與大眾的共鳴度
            </p>
        </div>
        """, unsafe_allow_html=True)
    
    with ex_col3:
        st.markdown("""
        <div style='background: #F3F4F6; padding: 1.5rem; border-radius: 10px; height: 100%;'>
            <h4 style='color: #1E3A8A; margin-top: 0;'>📊 學術研討會</h4>
            <p style='color: #6B7280; font-size: 0.9rem;'>
                檢視專業度、邏輯架構、時間掌控與問答準備
            </p>
        </div>
        """, unsafe_allow_html=True)

Writing streamlit_app.py


In [ ]:
# 4. 啟動 Streamlit（方案 A：簡潔版 - 使用 Colab 內建方法）

import subprocess
import time
from IPython.display import display, Javascript

print("🚀 啟動 AI 演講教練...")
print("=" * 60)

# 啟動 Streamlit
process = subprocess.Popen(
    ["streamlit", "run", "streamlit_app.py", "--server.port=8501", "--server.headless=true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.PIPE
)

# 等待啟動
time.sleep(5)

print("✅ 應用已成功啟動！")
print("\n🌐 請使用以下任一方式訪問：")
print("\n方式 1（推薦）：直接在 Colab 中使用")
print("   執行下一個格子，會在下方顯示應用界面")
print("\n方式 2：使用 ngrok 建立公開連結")
print("   執行「方案 B」的格子（需要 ngrok token）")
print("\n" + "=" * 60)

Starting Streamlit in background...
Your Public IP for LocalTunnel is:
35.204.40.217

Click the link below and enter the IP above:
⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://big-walls-fold.loca.lt
/tools/node/lib/node_modules/localtunnel/bin/lt.js:81
    throw err;
    ^

Error: connection refused: localtunnel.me:6571 (check your firewall settings)
    at Socket.<anonymous> (/tools/node/lib/node_modules/localtunnel/lib/TunnelCluster.js:52:11)
    at Socket.emit (node:events:524:28)
    at emitErrorNT (node:internal/streams/destroy:169:8)
    at emitErrorCloseNT (node:internal/streams/destroy:128:3)
    at process.processTicksAndRejections (node:internal/process/task_queues:82:21)

Node.js v20.19.0
⠙

In [ ]:
# 5 [最穩定] 使用 ngrok 建立公開連結
# ⚠️ 需要免費註冊 ngrok 帳號（30 秒完成）

# ========================================
# 🔧 設置區：請在這裡填入你的 ngrok token
# ========================================
# 
# 📝 如何取得 token（只需做一次）：
# 
# 步驟 1️⃣：註冊免費帳號
#   👉 https://dashboard.ngrok.com/signup
#   （可用 Google/GitHub 快速登入）
# 
# 步驟 2️⃣：複製你的 authtoken
#   👉 https://dashboard.ngrok.com/get-started/your-authtoken
#   （登入後會直接看到）
# 
# 步驟 3️⃣：填入下方 YOUR_TOKEN_HERE 的位置
# 
NGROK_TOKEN = "YOUR_TOKEN_HERE"  # ⬅️ 把 token 貼在這裡（保留引號）
# 
# ========================================

# 安裝 pyngrok
!pip install -q pyngrok

from pyngrok import ngrok
import time

print("🚀 正在建立公開連結...")
print("=" * 60)

# 檢查是否已設置 token
if NGROK_TOKEN == "YOUR_TOKEN_HERE":
    print("\n❌ 尚未設置 ngrok token！")
    print("\n📝 請依照上方「🔧 設置區」的三個步驟操作：")
    print("\n   1️⃣  註冊免費帳號：https://dashboard.ngrok.com/signup")
    print("   2️⃣  複製你的 authtoken：https://dashboard.ngrok.com/get-started/your-authtoken")
    print("   3️⃣  填入上方 NGROK_TOKEN = \"YOUR_TOKEN_HERE\" 的位置")
    print("\n💡 或者使用上一格（5A）的 iframe 方式，完全不需要 token！")
    print("=" * 60)
else:
    try:
        # 設置 authtoken
        ngrok.set_auth_token(NGROK_TOKEN)
        
        # 建立 ngrok tunnel
        public_url = ngrok.connect(8501)
        
        # 等待穩定
        time.sleep(2)
        
        print("\n✅ 公開連結已建立！")
        print("\n" + "=" * 60)
        print(f"🌐 請點擊連結訪問：")
        print(f"\n    {public_url}")
        print("\n" + "=" * 60)
        print("\n💡 提示：")
        print("   • 這個連結可以分享給其他人使用")
        print("   • 連結在同一個 session 內保持不變")
        print("   • 關閉 Colab 後連結會失效")
        print("=" * 60)
        
    except Exception as e:
        print(f"\n❌ 建立連結失敗：{e}")
        print("\n💡 請確認：")
        print("   1. Token 是否正確複製（沒有多餘空格）")
        print("   2. 是否已完成 email 驗證")
        print("   3. 重新取得 token：https://dashboard.ngrok.com/get-started/your-authtoken")